In [1]:
from kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'

kitti = KittiDataset(dataset_path, sequence)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
labeled_pcds = sm.get_combined_labeled_point_clouds(20, 25)

image 20 processed
image 21 processed
image 22 processed
image 23 processed
image 24 processed
